In [2]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stopwords

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/kritikseth/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


<WordListCorpusReader in '/Users/kritikseth/nltk_data/corpora/stopwords'>

In [8]:
stopwords = set(stopwords.words('english'))

In [7]:
stops

{'a',
 'about',
 'above',
 'after',
 'again',
 'against',
 'ain',
 'all',
 'am',
 'an',
 'and',
 'any',
 'are',
 'aren',
 "aren't",
 'as',
 'at',
 'be',
 'because',
 'been',
 'before',
 'being',
 'below',
 'between',
 'both',
 'but',
 'by',
 'can',
 'couldn',
 "couldn't",
 'd',
 'did',
 'didn',
 "didn't",
 'do',
 'does',
 'doesn',
 "doesn't",
 'doing',
 'don',
 "don't",
 'down',
 'during',
 'each',
 'few',
 'for',
 'from',
 'further',
 'had',
 'hadn',
 "hadn't",
 'has',
 'hasn',
 "hasn't",
 'have',
 'haven',
 "haven't",
 'having',
 'he',
 'her',
 'here',
 'hers',
 'herself',
 'him',
 'himself',
 'his',
 'how',
 'i',
 'if',
 'in',
 'into',
 'is',
 'isn',
 "isn't",
 'it',
 "it's",
 'its',
 'itself',
 'just',
 'll',
 'm',
 'ma',
 'me',
 'mightn',
 "mightn't",
 'more',
 'most',
 'mustn',
 "mustn't",
 'my',
 'myself',
 'needn',
 "needn't",
 'no',
 'nor',
 'not',
 'now',
 'o',
 'of',
 'off',
 'on',
 'once',
 'only',
 'or',
 'other',
 'our',
 'ours',
 'ourselves',
 'out',
 'over',
 'own',
 'r

In [73]:
# import datasets
# from datasets import load_dataset
# from transformers import AutoTokenizer
# from torch.utils.data import DataLoader
# from transformers import AutoModelForSequenceClassification
# from torch.optim import AdamW
# from transformers import get_scheduler
# import torch
# from tqdm.auto import tqdm
# import evaluate
import random
import argparse
from nltk.corpus import wordnet
from nltk import word_tokenize
from nltk.tokenize.treebank import TreebankWordDetokenizer
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stopwords = set(stopwords.words('english'))
random.seed(0)


def example_transform(example):
    example["text"] = example["text"].lower()
    return example


### Rough guidelines --- typos
# For typos, you can try to simulate nearest keys on the QWERTY keyboard for some of the letter (e.g. vowels)
# You can randomly select each word with some fixed probability, and replace random letters in that word with one of the
# nearest keys on the keyboard. You can vary the random probablity or which letters to use to achieve the desired accuracy.


### Rough guidelines --- synonym replacement
# For synonyms, use can rely on wordnet (already imported here). Wordnet (https://www.nltk.org/howto/wordnet.html) includes
# something called synsets (which stands for synonymous words) and for each of them, lemmas() should give you a possible synonym word.
# You can randomly select each word with some fixed probability to replace by a synonym.

def add_typing_error(sentence):
    key_neighbors = {
        'a': ['q', 's'], 'b': ['v', 'g'], 'c': ['x', 'd'], 'd': ['s', 'f'],
        'e': ['w', 'r'], 'f': ['d', 'g'], 'g': ['f', 'h'], 'h': ['g', 'j'],
        'i': ['u', 'o'], 'j': ['h', 'k'], 'k': ['j', 'l'], 'l': ['k', ';'],
        'm': ['n', ','], 'n': ['b', 'm'], 'o': ['i', 'p'], 'p': ['o', '['],
        'q': ['a', 'w'], 'r': ['e', 't'], 's': ['a', 'd'], 't': ['r', 'y'],
        'u': ['y', 'i'], 'v': ['c', 'b'], 'w': ['q', 'e'], 'x': ['z', 'c'],
        'y': ['t', 'u'], 'z': ['x', 's']
    }

    words = sentence.split()
    transformed_words = []

    for word in words:
        if random.random() <= 0.3 and len(word) > 2:
            for _ in range(random.randint(1, 2)):
                index = random.randint(0, len(word) - 1)
                original_letter = word[index]
                if original_letter.isalpha():
                    replacement_letter = random.choice(key_neighbors.get(original_letter, [original_letter]))
                    word = word[:index] + replacement_letter + word[index + 1:]

        transformed_words.append(word)

    return " ".join(transformed_words)

def replace_words_with_synonyms(sentence):
    words = sentence.split()
    transformed_words = []

    num_replacements = random.randint(1, 2)
    replacements_made = 0

    for word in words:
        if replacements_made < num_replacements and word not in stopwords:
            if random.random() <= 0.4:
                synonyms = get_synonyms(word)
                if synonyms:
                    transformed_words.append(random.choice(synonyms))
                    replacements_made += 1
                else:
                    transformed_words.append(word)
            else:
                transformed_words.append(word)
        else:
            transformed_words.append(word)

    return " ".join(transformed_words)

def get_synonyms(word):
    synonyms = []
    for syn in wordnet.synsets(word):
        for lemma in syn.lemmas():
            synonyms.append(lemma.name())
    return synonyms

def replace_preposition(sentence):

    prepositions = {
        "at": ["in", "on", "to", "for"],
        "in": ["into", "at", "on", "to",  "for"],
        "on": ["onto", "at", "in", "to", "for"],
        "to": ["at", "in", "on", "for"],
        "for": ["at", "in", "on", "to"]
    }

    words = sentence.split()
    transformed_words = []
    for word in words:
        if word.lower() in prepositions:
            replacement = random.choice(prepositions[word.lower()])
            transformed_words.append(replacement)
        else:
            transformed_words.append(word)
    
    return " ".join(transformed_words)

def replace_or_remove_articles(sentence):
    articles = ["a", "an", "the"]
    words = sentence.split()
    transformed_words = []

    for word in words:
        if word.lower() in articles:
            if random.random() <= 0.5:
                transformed_words.append("")
            else:
                incorrect_articles = [a for a in articles if a != word.lower()]
                replacement = random.choice(incorrect_articles)
                transformed_words.append(replacement)
        else:
            transformed_words.append(word)
    
    return " ".join(transformed_words)

def remove_be_forms(sentence):
    be_forms = ["am", "is", "are", "was", "were"]
    words = sentence.split()
    transformed_words = []

    for word in words:
        if word.lower() in be_forms:
            transformed_words.append("")
        else:
            transformed_words.append(word)
    
    return " ".join(transformed_words)

def custom_transform(example):
    ################################
    ##### YOUR CODE BEGINGS HERE ###

    # Design and implement the transformation as mentioned in pdf
    # You are free to implement any transformation but the comments at the top roughly describe
    # how you could implement two of them --- synonym replacement and typos.

    # You should update example["text"] using your transformation

    ###
    sentences = example["text"].split(".")
    transformed_sentences = []

    for sentence in sentences:
        transformed_sentence = replace_words_with_synonyms(sentence)
        transformed_sentence = add_typing_error(transformed_sentence)

        if random.random() <= 0.75:
            additional_transformations = [replace_preposition, replace_or_remove_articles, remove_be_forms]
            selected_transformations = random.sample(additional_transformations, random.randint(1, 3))

            for transformation in selected_transformations:
                transformed_sentence = transformation(transformed_sentence)

        transformed_sentences.append(transformed_sentence)

    example["text"] = ". ".join(transformed_sentences)
    ##### YOUR CODE ENDS HERE ######

    return example


[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/kritikseth/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [87]:
sentence = {}
sentence['text']= 'Stop words are commonly used words such as articles, pronouns and prepositions. Stop words are not added to the search dictionary, but they are counted as words for proximity (a distance between words) searching purposes'
custom_transform(sentence)

{'text': 'Stop eords are ordinarily used wrangle such as articles, pronoins abd prepositions. Srop Word not afded for an dwarch dixtionary, but thry counred as words in proximity (a distance beteewn wotss) searching pirposes'}

In [1]:
from datasets import load_dataset
dataset = load_dataset("imdb")

/Users/kritikseth/anaconda3/envs/luckyorgenius/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Generating unsupervised split: 100%|██████████| 50000/50000 [00:04<00:00, 10584.14 examples/s]


DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    unsupervised: Dataset({
        features: ['text', 'label'],
        num_rows: 50000
    })
})

In [3]:
import random